In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix as con
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import os

C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-pack

Instructions for updating:
non-resource variables are not supported in the long term


C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lab106\anaconda3\envs\tensorflow\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lab106\anaconda3\envs\ten

In [5]:
# network parameters
batch_size = 50  # 分批，每一批大小为100
epochs = 100
original_dim = 94  # 输入94维
intermediate_dim1 = 74  # 隐藏层1
intermediate_dim2 = 32  # 隐藏层2
latent_dim = 2 # 隐变量2维
out1_dim = 94  # 重构x
out2_dim = 3   # 分类y

# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling
        fr an isotropic unit Gaussian.
    # Arguments:
        args (tensor): mean and log of variance of Q(z|X)
    # Returns:
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    # K is the keras backend
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# VAE model = encoder + decoder
# build encoder model

x = Input(shape=(original_dim,), name='input') # 维数为original_dim（94），数量不确定
h1 = Dense(intermediate_dim1, activation='relu')(x)
h2 = Dense(intermediate_dim2, activation='relu')(h1)

# 算p(Z|X)的均值和方差
z_mean = Dense(latent_dim, name='z_mean')(h2)  # 隐层均值，输入为h，输出维数为2
z_log_var = Dense(latent_dim, name='z_log_var')(h2)  # 隐层方差，输入为h，输出维数为2


# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary
# with the TensorFlow backend
z = Lambda(sampling,
           output_shape=(latent_dim,),
           name='z')([z_mean, z_log_var])

decoder_h1 = Dense(intermediate_dim2, activation='relu')  # 32个神经元
decoder_h2 = Dense(intermediate_dim1, activation='relu')  # 74个神经元
decoder_mean = Dense(out1_dim, activation='sigmoid', name='output1')  # 94个神经元
decoder_y = Dense(out2_dim, activation='softmax', name='output2')

# h1_decoded = decoder_h1(z)
h1_decoded = decoder_h1(z)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
h2_decoded = decoder_h2(h1_decoded)
x_decoded = decoder_mean(h2_decoded)
y_clf = decoder_y(x_decoded)

for L in range(5, 55, 5):
    for a in range(5, 55, 5):
#       print(L, a)
        M=a*L
        H=a*a*L
        cost=[[1,H,H],
              [M,1,M],
              [L,L,1]]

        def loss1(x, x_decoded):    
            xent_loss = K.sum(K.binary_crossentropy(tf.convert_to_tensor(x), tf.convert_to_tensor(x_decoded)), axis=-1) # 重构误差
            kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) # KL散度
            vae_loss = xent_loss + kl_loss 
            return vae_loss

        def loss2(y_true, y_pred):
#           clf_loss = -tf.reduce_mean(tf.convert_to_tensor(y_true) * tf.log(tf.convert_to_tensor(y_pred))) * cost[np.argmax(y_true)][np.argmax(y_pred)]
            clf_loss = -tf.reduce_mean(tf.convert_to_tensor(y_true) * tf.log(tf.convert_to_tensor(y_pred))) 
#           clf_loss = -tf.reduce_mean(np.argmax(y_true) * tf.log(np.argmax(y_pred)) * cost[np.argmax(y_true)][np.argmax(y_pred)]
            return clf_loss


        vae=Model(inputs = x, outputs = [x_decoded, y_clf])
        vae.compile(optimizer='adam',
                    loss = {'output1':loss1,'output2':loss2},
                    loss_weights = {'output1':1.0,'output2':100})


        data = pd.read_csv("Full_Dataset-Dmax-TTT.csv")
        # 打乱数据顺序
        data = shuffle(data)
        # shuffle(data)
        # print(data.iloc[0:1,0:95])
        X = data.iloc[0:5935,0:94]
        min_max_scaler = preprocessing.MinMaxScaler()
        X = min_max_scaler.fit_transform(X)
        Y = pd.get_dummies(data['Phase Formation'])
        Y = Y.values

        KF = KFold(n_splits = 10,shuffle=True,random_state=100)

        sumCon = [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]]

        sumAccuracy = 0

        for train_index, test_index in KF.split(X):
#     print('\n',i)
#     print("TRAIN:", train_index, "TEST:", test_index)
            x_train, x_test = X[train_index], X[test_index]
            y_train, y_test = Y[train_index], Y[test_index]
#     print('训练集数量:',x_train.shape,'测试集数量:',x_test.shape) #结果表明每次划分的数量
#     print(y_train)
            vae.fit({'input':x_train}, 
                    {'output1':x_train,'output2':y_train}, 
                    shuffle=True,
                    epochs=epochs,
                    verbose=0,
                    batch_size=batch_size)
    
            y1, y2 = vae.predict(x_test)
    
            y_predict=[]
            for i in y2:
                y_predict.append(np.argmax(i))
#               print(y_predict)

            y_true =[]
            for j in y_test:
                y_true.append(np.argmax(j))
#               print(y_true)

            print('混淆矩阵：')
            print(con(y_true,y_predict))
            sumCon += con(y_true,y_predict)
    
            print('Accuracy:')
            print(accuracy_score(y_true, y_predict))
            sumAccuracy += accuracy_score(y_true, y_predict)
#             i += 1

        print('\n')
        print(L, a)
        print(sumCon)
        print(sumAccuracy / 10.0)

    



10 5
[[ 635    0   40]
 [   3 1241  308]
 [  42  291 3375]]
0.884767290669483


10 10
[[ 643    1   31]
 [   7 1270  275]
 [  19  167 3522]]
0.9157593359111067


10 15
[[ 653    0   22]
 [   2 1289  261]
 [  23  116 3569]]
0.9285667240136043


10 20
[[ 649    2   24]
 [   2 1380  170]
 [  12  132 3564]]
0.9423802385859721


10 25
[[ 649    2   24]
 [   4 1392  156]
 [  20  143 3545]]
0.9411946900142516


10 30
[[ 655    4   16]
 [   2 1401  149]
 [  12  129 3567]]
0.9474335825937852


10 35
[[ 659    3   13]
 [   4 1395  153]
 [  13  110 3585]]
0.9501274691831185


10 40
[[ 653    2   20]
 [   1 1430  121]
 [  15  117 3576]]
0.9534956081330449


10 45
[[ 653    0   22]
 [   3 1426  123]
 [   7  113 3588]]
0.9548480873944618


10 50
[[ 660    0   15]
 [   2 1458   92]
 [  13  103 3592]]
0.9620911191737498


15 5
[[ 655    2   18]
 [   2 1449  101]
 [   9   99 3600]]
0.9610790308935332


15 10
[[ 656    2   17]
 [   3 1429  120]
 [   7   98 3603]]
0.9583842926170076


15 15
[[ 656    3

KeyboardInterrupt: 